In [2]:
# imports
%matplotlib inline
import pandas
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import time
import datetime
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from estatisticas_time_serie import generate_datasets,readData,general_forecasting,smooth, lstm_predictions

from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')



Using TensorFlow backend.


ImportError: No module named tensorflow

In [ ]:
sensores = [
    '690a-vila-aurea-4meses.csv',
    '690g-pressao-vila-aurea-4meses.csv',
    '605g-pressaoalta-vila-aurea-4meses.csv',
    '735g-pressaomedia-vila-aurea-4meses.csv',
    '450a-vazao-santa-izabel-4meses.csv',
    '450g-pressao_entrada-santa-izabel-4meses.csv',
    '485b-vazao-reno-4meses.csv',
    '485g-pressao_entrada-reno-4meses.csv',
    '525g-pressao_baixa-reno-4meses.csv',
    '573g-pressao_alta-aeroporto-4meses.csv',
    '589g-pressao_alta-andreazza-4meses.csv',
    '631g-pressao_ponto_critico-andreazza-4meses.csv',
    '635g-pressao_ponto_critico-aeroporto-4meses.csv',
    '678b-vazao-andreazza-4meses.csv',
    '678g-pressao_entrada-andreazza-4meses.csv',
    '781a-vazao-aeroporto-4meses.csv',
    '781g-pressao_entrada-aeroporto-4meses.csv',
    '801g-pressao_alta-reno-4meses.csv',
    '830g-pressao_baixa-santa-izabel-4meses.csv',
    '831g-pressao_alta-santa-izabel-4meses.csv'
]

titulos = [
    'Vila aurea vazao',
    'Vila aurea pressao entrada',
    'Vila aurea pressao alta',
    'Vila aurea pressao media',
    'Santa Izabel vazao',
    'Santa Izabel pressao entrada',
    'Reno vazao',
    'Reno pressao entrada',
    'Reno pressao baixa',
    'Aeroporto pressao alta',
    'Andreazza pressao ponto critico',
    'Aeroporto pressao ponto critico',
    'Andreazza vazao',
    'Andreazza pressao entrada',
    'Andreazza vazao',
    'Aeroporto vazao',
    'Aeroporto pressao entrada',
    'Reno pressao alta',
    'Santa Izabel pressao baixa',
    'Santa Izabel pressao alta'
]

dataframes = []
for index, item in enumerate(sensores):
    # load the first dataset vilacarvalho-vazao-60
    dataframe = pandas.read_csv(item, engine='python', skipfooter=3)
    data_matrix = dataframe.as_matrix()
    data_matrix = data_matrix[:,2:]

    data_convolution = np.array(data_matrix).ravel()

    newV=[]
    for idx,val in enumerate(data_convolution):
        if not data_convolution[idx] == 'null' and not np.isnan(float(data_convolution[idx])) and float(data_convolution[idx]) > 0.0: 
            newV.append(float(data_convolution[idx]))

    newV = np.array(newV)
    def reject_outliers(data, m=2):
        return data[abs(data - np.mean(data)) < m * np.std(data)]
    newV = reject_outliers(newV)
    dataframes.append(newV)

    plt.figure(num=None, figsize=(12, 4), dpi=80, facecolor='w', edgecolor='k')
    plt.plot(newV,label='values')
    plt.title(titulos[index])
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()

In [ ]:
for item in dataframes:
    scaler = MinMaxScaler(feature_range=(0, 1))
    # normalize the dataset
    datasetLstm = scaler.fit_transform(item)
    train_data_lstm, test_data_lstm = generate_datasets(datasetLstm,look_back = 48,method='retro')

    trainXlstm, trainYlstm = train_data_lstm[0], train_data_lstm[1]
    testXlstm, testYlstm   = test_data_lstm[0], test_data_lstm[1]
    # reshape input to be [samples, time steps, features]
    trainXlstm = np.reshape(trainXlstm, (trainXlstm.shape[0], 1, trainXlstm.shape[1]))
    testXlstm = np.reshape(testXlstm, (testXlstm.shape[0], 1, testXlstm.shape[1]))

    print("training lstm")
    #LSTM
    model = Sequential()
    model.add(LSTM(10, input_dim=48))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    model.fit(trainXlstm, trainYlstm, nb_epoch=30, batch_size=128, verbose=0)

    forecast_LSTM = general_forecasting(testXlstm, model,method='lstm',size_forecasting=48, strategy_approach='retro')
    # rescaling back to initial dimension
    forecast_LSTM = scaler.inverse_transform(forecast_LSTM)
    testY_lstm = scaler.inverse_transform(testYlstm)

    print("mean_squared_error  LSTM = ",mean_squared_error(testY_lstm[:48],forecast_LSTM))

    #plotting lstm results
    testPredictPlot = np.empty((int(newV.shape[0])))
    testPredictPlot[:] = np.nan
    testPredictPlot = forecast_LSTM

    plt.figure(num=None, figsize=(12, 4), dpi=80, facecolor='w', edgecolor='k')
    plt.plot(testY_lstm[:48],color='b',label='target test')
    plt.plot(testPredictPlot[:48], color='r', label='predicted')
    plt.title('predicting with LSTM using 4 months')
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    plt.show()